In [1]:
import glob
from langchain_community.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.schema import Document
import pymupdf
import os

In [2]:
# PDF dosyasını yükleme
pdf_folders_path =glob.glob("knowledge_base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
print(pdf_folders_path)


['knowledge_base/3GPP', 'knowledge_base/articles']


In [3]:
chunked_documents = []

for pdf_folder in pdf_folders_path:
    
    print(f"Processing folder: {pdf_folder}")

    loader = DirectoryLoader(pdf_folder, glob="**/*.pdf", loader_cls=PyMuPDFLoader)
    folder_docs = loader.load()


    pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))


    for pdf_path in pdf_files:
        # PDF dosyasını aç
        pdf_open = pymupdf.open(pdf_path)
        
        # Table of Contents (TOC) al
        toc = pdf_open.get_toc()

        for i, item in enumerate(toc):
            heading = item[1]
            start_page = item[2]

            if i + 1 < len(toc):
                end_page = toc[i + 1][2] - 1
            else:
                end_page = pdf_open.page_count  # Son başlıksa PDF'in son sayfasına kadar
    
            # Sonraki başlığın sayfasına kadar olan kısmı almak
            if i + 1 < len(toc):
                end_page = toc[i + 1][2] - 1
            else:
                end_page = pdf_open.page_count -  1   # Son başlıksa PDF'in son sayfasına kadar

            # Metni birleştirerek chunk oluştur
            chunk_text = ""
            for page_num in range(start_page, end_page):  # PyMuPDF için 0-index
                chunk_text += pdf_open[page_num].get_text()

        # İlk chunk'ları oluştur ve listeye ekle
            chunked_documents.append(
                Document(
                    page_content=chunk_text,
                    metadata={"heading": heading, "start_page": start_page, "end_page": end_page}
                )
            )


Processing folder: knowledge_base/3GPP
Processing folder: knowledge_base/articles


In [4]:
# CharacterTextSplitter kullanarak ek parçalama
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# TOC'ye göre bölünmüş metinleri tekrar split ediyoruz
final_chunks = text_splitter.split_documents(chunked_documents)

# Çıktıyı kontrol etmek için

In [5]:
len(chunked_documents)

1072

In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

# Create our Chroma vectorstore!
db_name = "vector-db"

In [12]:
import torch
from langchain_chroma import Chroma
import os
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embeddings = FastEmbedEmbeddings(device="cpu")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=final_chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1416 documents


In [13]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from tiktoken import get_encoding
from langchain.schema import HumanMessage
#from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [14]:
GRADER_PROMPT_TEMPLATE = """
You are an expert evaluator for RAG systems. Evaluate the response using the following criteria:

1. **Groundedness**: 
   - Does the response rely on and accurately use the provided context? 
   - A high score (close to 1) means the response is fully grounded in the given context, while a low score (close to 0) indicates that the response includes information that is not present or contradicts the context. Your score should be between 0 and 1. Such as 0.45.

2. **Answer Relevance**: 
   - Is the response relevant to and directly answering the question?
   - A high score (close to 1) means the response is precise, well-aligned with the question, and provides a clear answer. A low score (close to 0) indicates the response is irrelevant or off-topic.Your score should be between 0 and 1. Such as 0.45.

3. **Context Relevance**: 
   - Does the provided context contain information that is useful and directly related to answering the question?
   - A high score (close to 1) means the context fully supports answering the question, while a low score (close to 0) means the context is unrelated or insufficient for the question.Your score should be between 0 and 1. Such as 0.45.


------

Evaluate the following inputs:

**Context**:  
{context}

**Question**:  
{question}

**Response**:  
{response}

**Ground Truth**:  
{ground_truth}

------

Provide a float score between 0 and 1 such as 0.534 for each criterion along with a short justification for your score. The output should follow this format:

Groundedness: X/1  
Explanation: [Your explanation]

Answer Relevance: X/1 
Explanation: [Your explanation]

Context Relevance: X/1  
Explanation: [Your explanation]

"""

In [17]:
questions_answers = [
    {
        "question": "Can you categorize large Telecom model use cases?",
        "answer": 
            "1. Network Management and Optimization:\n"
            "- Qiming Network Model: Supports network planning, maintenance, monitoring, troubleshooting, and performance optimization. Enhances real-time decision-making processes.\n"
            "- Network Automation & Intent-Based Management: Translates user intents into machine-readable commands to automate network operations.\n\n"
            
            "2. Physical and MAC Layer Design:\n"
            "- Wireless Signal Analysis: Optimizes tasks like spectrum management and interference detection using time-series data.\n\n"
            
            "3. Data Management and Security:\n"
            "- Fraud Detection & Security: Utilizes transfer learning for anomaly detection to identify fraudulent activities.\n"
            "- Data Compliance: Ensures data processing workflows align with regulations like GDPR.\n\n"
            
            "4. Vertical Applications:\n"
            "- Healthcare: Supports remote patient monitoring and predictive diagnostics.\n"
            "- Smart Cities: Applied in traffic management, energy optimization, and public safety initiatives.\n\n"
            
            "5. Next-Generation Networks (5G and Beyond):\n"
            "- Network Slicing & Edge Computing: Optimizes resource allocation to deliver more flexible services.\n"
            "- Telecom Co-Pilots: Develops intelligent assistants for operational support."
        
    }
]


In [18]:
import json
import requests
import gradio as gr

# OLLAMA API URL
url = "http://45.145.22.22:11434/api/chat"

# Başlıklar
headers = {
    "Content-Type": "application/json"
}

# Retriever - VectorStore
retriever = vectorstore.as_retriever()

# API'den yanıt ve değerlendirme almak için fonksiyon tanımlama
def ask_and_evaluate(user_question, ground_truth):
    # similarity_search çıktısını al
    retrieved_contexts = vectorstore.similarity_search(user_question, k=10)

    # Belgelerin içeriğini birleştir
    combined_context = " ".join([doc.page_content for doc in retrieved_contexts])

    # API ile iletişim için mesajı hazırla
    input_message = {
        "model": "llama3.2",
        "messages": [
            {"role": "system", "content": "You are a telecom assistant."},
            {"role": "user", "content": f"Context: {combined_context}\n\nQuestion: {user_question}"}
        ]
    }

    # API'ye istek gönder ve yanıtı oku
    response = requests.post(url, json=input_message, headers=headers)

    if response.status_code == 200:
        combined_response = ""
        for line in response.text.strip().split("\n"):
            try:
                data = json.loads(line)
                content = data['message']['content']
                combined_response += content
            except json.JSONDecodeError as e:
                print(f"JSON Decode Hatası: {e}")
                continue
    else:
        return f"API Hatası: {response.status_code} - {response.text}", "Evaluation Error"

    # Değerlendirme prompt'unu oluştur
    grader_prompt = GRADER_PROMPT_TEMPLATE.format(
        context=combined_context,
        question=user_question,
        response=combined_response,
        ground_truth=ground_truth
    )

    # Değerlendirme için API'yi çağır
    evaluation_response = requests.post(url, json={"model": "llama3.2", "messages": [{"role": "user", "content": grader_prompt}]}, headers=headers)

    if evaluation_response.status_code == 200:
        eval_combined_response = ""
        for line in evaluation_response.text.strip().split("\n"):
            try:
                eval_data = json.loads(line)
                eval_content = eval_data['message']['content']
                eval_combined_response += eval_content
            except json.JSONDecodeError as e:
                print(f"Değerlendirme JSON Decode Hatası: {e}")
                continue
    else:
        eval_combined_response = f"Değerlendirme API Hatası: {evaluation_response.status_code} - {evaluation_response.text}"

    return combined_response, eval_combined_response

# Gradio arayüzü
with gr.Blocks() as demo:
    gr.Markdown("## Telecom Assistant QA and Evaluation")
    question_input = gr.Textbox(label="Soru", placeholder="Sorunuzu buraya yazın...")
    ground_truth_input = gr.Textbox(label="Ground Truth", placeholder="Enter the expected ground truth...")
    submit_btn = gr.Button("Sor ve Değerlendir")

    response_output = gr.Textbox(label="Model Yanıtı")
    evaluation_output = gr.Textbox(label="Değerlendirme Sonucu")

    submit_btn.click(ask_and_evaluate, inputs=[question_input, ground_truth_input], outputs=[response_output, evaluation_output])

# Arayüzü başlatma
demo.launch(share= True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://8d50847a12ae6b33c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
import gradio as gr
iface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Bir soru girin..."),
    outputs=gr.Textbox(label="Yanıt"),
    title="Telecom Assistant",
    description="Telekom özelinde sorularınızı sorabilirsiniz."

)


# Uygulamayı başlat
iface.launch(share=True)


NameError: name 'ask_question' is not defined

In [24]:
    print(f"Question:{question}")
    print(f"Response:{combined_response}")
    print(f"Ground Truth:{ground_truth}")
    print(f"Context:{eval_combined_response}")
    print("\n\n\n\n")

Question:Can you categorize large Telecom model use cases?
Response:The use cases for Large Telecom Models can be categorized into the following:

**Application Enhancement**

1. Improving network capabilities by adding new functionalities or improving existing ones.
2. Enhancing mobile network operations through automation and optimization.

**Network Operations and Maintenance**

3. Automating network configuration generation.
4. Reducing manual effort in network management.
5. Improving network performance, security, and compliance.

**Customer Service and Business Processes**

6. Developing network-related software components.
7. Generating troubleshooting solutions.
8. Providing zero-shot image classification capabilities for complicated signal transmission environments.

**Policy Generation and Intent-Based Application Management**

9. Leveraging LLMs for policy generation.
10. Automating intent-based application management.

**Troubleshooting and Fault Diagnosis**

11. Diagnosin

from transformers import AutoModelForCausalLM, AutoTokenizer

# Model ve Tokenizer'ı yükle
model_name = "/mnt/storage/Llama-3.3-70B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Retriever - VectorStore
retriever = vectorstore.as_retriever()

# Sonuçları saklamak için bir liste
evaluation_results = []

# Her bir input üzerinde işlem yapma
for qa_pair in questions_answers:
    question = qa_pair["question"]
    ground_truth = qa_pair["answer"]

    # similarity_search çıktısını al
    retrieved_contexts = vectorstore.similarity_search(question, k=10)
    
    # Belgelerin içeriğini birleştir
    combined_context = " ".join([doc.page_content for doc in retrieved_contexts])

    # Input mesajını hazırlayın
    input_text = f"Context: {combined_context}\n\nQuestion: {question}"

    # Input verisini token'lara dönüştür
    inputs = tokenizer(input_text, return_tensors="pt")

    # Modelden yanıt al
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1)

    # Yanıtı çözümle
    model_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Değerlendirme prompt'unu oluştur
    grader_prompt = GRADER_PROMPT_TEMPLATE.format(
        context=" ".join([doc.page_content for doc in retrieved_contexts]),
        question=question,
        response=model_response,
        ground_truth=ground_truth
    )

    # Değerlendirme için modelden yanıt al
    evaluation_inputs = tokenizer(grader_prompt, return_tensors="pt")
    evaluation_outputs = model.generate(**evaluation_inputs, max_length=512, num_return_sequences=1)

    # Değerlendirme yanıtını çözümle
    evaluation_result = tokenizer.decode(evaluation_outputs[0], skip_special_tokens=True)

    # Sonuçları kaydet
    evaluation_results.append({
        "query": question,
        "response": model_response,
        "ground_truth": ground_truth,
        "evaluation": evaluation_result
    })



# Memory
#memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Retriever - VectorStore 
retriever = vectorstore.as_retriever()

# Conversational Retrieval Chain
#conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

llm = ChatOllama(
                 model = "llama3.2")
# Sonuçları saklamak için bir liste
evaluation_results = []

# Her bir input üzerinde işlem yapma

for qa_pair in questions_answers:
    question = qa_pair["question"]
    ground_truth = qa_pair["answer"]
    # similarity_search çıktısını stringe dönüştür
    retrieved_contexts = vectorstore.similarity_search(question, k=10)
    combined_context = " ".join([doc.page_content for doc in retrieved_contexts])

    
    input_message = HumanMessage(content=f"Context: {combined_context}\n\nQuestion: {question}")
    response = llm.invoke([input_message])
    # Değerlendirme prompt'unu hazırlama
    grader_prompt = GRADER_PROMPT_TEMPLATE.format(
                    context=retrieved_contexts,
                    question=question,
                    response=response.content,
                    ground_truth=ground_truth)
        
    evaluation_result = llm.invoke(grader_prompt)

    evaluation_results.append({
        "query": question,
        "response": response,
        "ground_truth": ground_truth,
        "evaluation": evaluation_result.content
    })



In [14]:
import torch


In [ ]:
# Memory
#memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Retriever - VectorStore 
retriever = vectorstore.as_retriever()

# Conversational Retrieval Chain
#conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

llm = ChatOllama(
                 model = "llama3.2")
# Sonuçları saklamak için bir liste
evaluation_results = []

# Her bir input üzerinde işlem yapma

for qa_pair in questions_answers:
    question = qa_pair["question"]
    ground_truth = qa_pair["answer"]
    # similarity_search çıktısını stringe dönüştür
    retrieved_contexts = vectorstore.similarity_search(question, k=10)
    combined_context = " ".join([doc.page_content for doc in retrieved_contexts])

    
    input_message = HumanMessage(content=f"Context: {combined_context}\n\nQuestion: {question}")
    response = llm.invoke([input_message])
    # Değerlendirme prompt'unu hazırlama
    grader_prompt = GRADER_PROMPT_TEMPLATE.format(
                    context=retrieved_contexts,
                    question=question,
                    response=response.content,
                    ground_truth=ground_truth)
        
    evaluation_result = llm.invoke(grader_prompt)

    evaluation_results.append({
        "query": question,
        "response": response,
        "ground_truth": ground_truth,
        "evaluation": evaluation_result.content
    })



In [115]:
    print(f"Question:{question}")
    print(f"Response:{response}")
    print(f"Ground Truth:{ground_truth}")
    print(f"Context:{evaluation_result.content}")
    print("\n\n\n\n")

Question:What is the significance of TreselectionNR, and how does it impact cell reselection?
Response:content="T-ReselectionNR (also known as treselection) is a parameter used to determine when a UE (User Equipment) should perform reselection to another cell. It is part of the 5G New Radio (NR) protocol.\n\nIn the context of NR, treselection refers to the process by which the UE evaluates new cells for reselection based on specific criteria, such as signal strength and priority. The T-ReselectionNR parameter specifies how quickly the UE should perform this evaluation after detecting a change in cell conditions or when the UE is experiencing a significant drop in signal quality.\n\nThe significance of treselectionNR lies in its impact on cell reselection:\n\n1. **Improved network performance**: By performing reselection more frequently, the UE can detect and switch to better-quality cells, leading to improved overall network performance.\n2. **Reduced handovers**: Frequent reselections

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")

llm.invoke("Tell me a joke")

KeyboardInterrupt: 

# TABLE EXTRACTION 

# IMAGE EXTRACTION

In [8]:





chunked_documents = []

for pdf in pdf_folders_path:
    loader = DirectoryLoader(pdf, glob="**/*.pdf", loader_cls=PyMuPDFLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()

    for pdf_ in folder_docs:
        open_pdf = pymupdf.open(pdf_)
        toc = open_pdf.get_toc()
        for i, item in enumerate(toc):
            heading = item[1]
            start_page = item[2]

            if i + 1 < len(toc):
                end_page = toc[i+1][2] - 1
            else:
                end_page = pdf.page_count

            chunk_text = ""

            for page_num in range(start_page -1, end_page):
                chunk_text += open_pdf[page_num].get_text()

            chunked_documents.append(
                Document(
                    page_content=chunk_text,
                    metadata={"heading": heading, "start_page": start_page, "end_page": end_page}
                )
            )
        open_pdf.close()        
    

    

"""
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()
pdf = pymupdf.open(pdf_path)

# TOC'yi al
toc = pdf.get_toc()

# LangChain için yeni Document objeleri oluşturma
chunked_documents = []

for i, item in enumerate(toc):
    heading = item[1]
    start_page = item[2]
    
    # Sonraki başlığın sayfasına kadar olan kısmı almak
    if i + 1 < len(toc):
        end_page = toc[i + 1][2] - 1
    else:
        end_page = pdf.page_count  # Son başlıksa PDF'in son sayfasına kadar

    # Metni birleştirerek chunk oluştur
    chunk_text = ""
    for page_num in range(start_page - 1, end_page):  # PyMuPDF için 0-index
        chunk_text += pdf[page_num].get_text()

    # İlk chunk'ları oluştur ve listeye ekle
    chunked_documents.append(
        Document(
            page_content=chunk_text,
            metadata={"heading": heading, "start_page": start_page, "end_page": end_page}
        )
    )
"""



Error loading file example/About-GSM/36201-i00.pdf


TypeError: get_text() got an unexpected keyword argument 'encoding'

In [9]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

# Create our Chroma vectorstore!
db_name = "vector-db"

In [ ]:

from langchain_chroma import Chroma
import os
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=final_chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
from langchain_ollama import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from tiktoken import get_encoding
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# LLM için OpenAI GPT 4O modelini tanımlıyoruz

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
llmOpenAI = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

# Memory
#memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Retriever - VectorStore 
retriever = vectorstore

# Conversational Retrieval Chain
#conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

# Değerlendirme prompt şablonu
GRADER_PROMPT_TEMPLATE = """
You are an expert evaluator for RAG systems. Evaluate the response using the following criteria:

1. **Groundedness**: 
   - Does the response rely on and accurately use the provided context? 
   - A high score (close to 1) means the response is fully grounded in the given context, while a low score (close to 0) indicates that the response includes information that is not present or contradicts the context. Your score should be between 0 and 1. Such as 0.45.

2. **Answer Relevance**: 
   - Is the response relevant to and directly answering the question?
   - A high score (close to 1) means the response is precise, well-aligned with the question, and provides a clear answer. A low score (close to 0) indicates the response is irrelevant or off-topic.Your score should be between 0 and 1. Such as 0.45.

3. **Context Relevance**: 
   - Does the provided context contain information that is useful and directly related to answering the question?
   - A high score (close to 1) means the context fully supports answering the question, while a low score (close to 0) means the context is unrelated or insufficient for the question.Your score should be between 0 and 1. Such as 0.45.

4. **Accuracy Compared to Ground Truth**:
   - How close is the response to the ground truth answer provided? 
   - A high score (close to 1) means the response matches the ground truth closely, while a low score (close to 0) indicates significant deviations.Your score should be between 0 and 1. Such as 0.45.

---

Evaluate the following inputs:

**Context**:  
{context}

**Question**:  
{question}

**Response**:  
{response}

**Ground Truth**:  
{ground_truth}

---

Provide a float score between 0 and 1 such as 0.534 for each criterion along with a short justification for your score. The output should follow this format:

Groundedness: X/1  
Explanation: [Your explanation]

Answer Relevance: X/1 
Explanation: [Your explanation]

Context Relevance: X/1  
Explanation: [Your explanation]

Accuracy Compared to Ground Truth: X/1  
Explanation: [Your explanation]
"""

MAX_TOKENS = 4096

def truncate_to_token_limit(text, token_limit=MAX_TOKENS, encoding_name="cl100k_base"):
    encoding = get_encoding(encoding_name)
    tokens = encoding.encode(text)
    if len(tokens) > token_limit:
        truncated_tokens = tokens[:token_limit]
        return encoding.decode(truncated_tokens)
    return text


questions_answers = [
    {
        "question": "What are the differences between the RRC_IDLE and RRC_INACTIVE states in terms of UE behavior?",
        "answer": "In RRC_IDLE, the UE camps on a cell, monitors system information, and performs paging while not connected. In RRC_INACTIVE, the UE maintains a context with the network but does not actively transmit or receive, allowing faster resumption of connection. Both states share procedures like PLMN selection and cell reselection."
    },
    {
        "question": "How does the UE determine a 'suitable cell' for camping? Which parameters does it evaluate?",
        "answer": "A 'suitable cell' is determined based on criteria like signal strength (Srxlev), signal quality (Squal), and whether the cell belongs to the selected PLMN or SNPN. The cell must not be barred or restricted."
    },
    {
        "question": "What are the procedures the UE follows during PLMN selection in both automatic and manual modes?",
        "answer": "In automatic mode, the UE searches for and selects a PLMN based on pre-configured priority lists or previously stored information. In manual mode, the user selects the PLMN from a list of available networks provided by the UE after scanning."
    },
    {
        "question": "Explain the difference between 'acceptable cells' and 'suitable cells.'",
        "answer": "Acceptable cells allow limited services, such as emergency calls and ETWS/CMAS notifications. Suitable cells support full network services and meet stricter criteria like being part of the registered or selected PLMN."
    },
    {
        "question": "How does the UE handle cell reselection priorities during inter-RAT transitions?",
        "answer": "The UE follows priority lists provided in system information or dedicated signaling. Frequencies with higher priorities are evaluated first, and measurements are conducted accordingly. Equal-priority cells are ranked based on signal strength and quality."
    },
    {
        "question": "What parameters affect cell reselection in the presence of high mobility?",
        "answer": "Parameters like TCRmax, NCR_H, and speed-dependent scaling factors modify reselection timers (e.g., TreselectionNR) and hysteresis (Qhyst). In high-mobility states, scaling reduces reselection delays to ensure seamless transitions."
    },
    {
        "question": "Describe the relaxed measurement rules for cell reselection. When are they applied?",
        "answer": "Relaxed measurement rules apply when the UE has low mobility or is not at the cell edge. Parameters like SSearchThresholdP and SSearchThresholdQ allow fewer measurements to conserve resources."
    },
    {
        "question": "What is the role of CAG cells in the 5G system, and how does the UE select them?",
        "answer": "CAG (Closed Access Group) cells are used for restricted access in private or enterprise networks. The UE selects these cells based on NAS-provided Allowed CAG lists and verifies that the CAG-ID matches the broadcast system information."
    },
    {
        "question": "How does the UE process paging messages in RRC_IDLE versus RRC_INACTIVE?",
        "answer": "In RRC_IDLE, the UE listens for paging in all tracking areas where it is registered. In RRC_INACTIVE, paging is restricted to a configured RNA (RAN-based Notification Area), reducing power consumption."
    },
    {
        "question": "What is the significance of TreselectionNR, and how does it impact cell reselection?",
        "answer": "TreselectionNR is a timer that defines the duration a target cell must meet reselection criteria before the UE reselects. It ensures stability by preventing rapid and unnecessary reselection events."
    }

]

"""
questions_answers = [
    {
        "question": "What is the impact of RNA updates in RRC_INACTIVE state?",
        "answer": "RNA updates help maintain UE's context within a specific area, reducing signaling overhead and improving power efficiency."
    },
    {
        "question": "How does the UE prioritize reselection between inter-frequency and intra-frequency cells?",
        "answer": "Intra-frequency cells are prioritized if they fulfill the reselection criteria. Inter-frequency cells are considered if they have higher priority and better signal quality."
    },
    {
        "question": "What role do hysteresis parameters play in cell reselection?",
        "answer": "Hysteresis parameters prevent frequent reselection by adding thresholds, ensuring stability in cell connections."
    },
    {
        "question": "How does the UE handle paging in a multi-PLMN environment?",
        "answer": "The UE listens for paging messages in all tracking areas associated with the selected PLMNs, optimizing response times."
    },
    {
        "question": "What criteria does the UE use to evaluate sidelink communication suitability?",
        "answer": "The UE evaluates parameters like signal strength, quality, and availability of sidelink configuration in system information broadcasts."
    },
    {
        "question": "What happens if a UE fails to decode system information on a selected cell?",
        "answer": "The UE continues searching for another suitable or acceptable cell to camp on, ensuring network availability."
    },
    {
        "question": "How are measurement gaps used to enhance cell reselection accuracy?",
        "answer": "Measurement gaps allow the UE to perform inter-frequency measurements without affecting its ongoing communication, improving reselection decisions."
    },
    {
        "question": "What is the role of PLMN identity in cell reselection?",
        "answer": "PLMN identity ensures the UE selects cells belonging to its registered or equivalent networks, maintaining seamless connectivity."
    },
    {
        "question": "How does the UE handle access control restrictions during cell selection?",
        "answer": "The UE checks access control parameters like CAG identifiers and avoids restricted cells, ensuring compliance with network policies."
    },
    {
        "question": "What is the significance of RSRP thresholds in reselection criteria?",
        "answer": "RSRP thresholds define minimum signal strength requirements for a cell to be considered suitable, ensuring reliable connectivity."
    },
    {
        "question": "Why does the UE store frequency information during cell reselection?",
        "answer": "Stored frequency information helps the UE expedite future reselection processes by avoiding redundant scans."
    },
    {
        "question": "What are the advantages of DRX cycles for paging in idle mode?",
        "answer": "DRX cycles reduce power consumption by allowing the UE to periodically wake up for paging checks, balancing responsiveness and efficiency."
    },
    {
        "question": "How does the UE perform reselection when multiple cells have similar priority?",
        "answer": "The UE ranks cells based on signal strength and quality, selecting the best cell among those with similar priority."
    }]
"""
"""
questions_answers = [
    {
        "question": "What happens during a tracking area update in the RRC_IDLE state?",
        "answer": "The UE informs the network of its new location, ensuring accurate paging and network communication."
    },
    {
        "question": "How do reselection timers impact the UE's mobility handling?",
        "answer": "Reselection timers like TreselectionNR delay transitions, ensuring the UE does not frequently switch cells unnecessarily."
    },
    {
        "question": "What role do SIB messages play in guiding UE behavior?",
        "answer": "System Information Blocks (SIBs) provide critical configuration data like reselection parameters, enabling the UE to make informed decisions."
    },
    {
        "question": "How does the UE use signal quality metrics like RSRQ during reselection?",
        "answer": "RSRQ evaluates the quality of the received signal, complementing signal strength metrics to ensure optimal reselection decisions."
    },
    {
        "question": "What is the significance of periodic searches for higher-priority PLMNs?",
        "answer": "Periodic searches help the UE identify better networks, ensuring improved service quality and connectivity."
    },
    {
        "question": "How does the UE handle reselection when operating in a shared spectrum?",
        "answer": "The UE evaluates shared spectrum cells using additional metrics like load balancing and interference, optimizing reselection."
    },
    {
        "question": "What triggers the UE to perform a cell reselection evaluation?",
        "answer": "Triggers include changes in signal strength or quality, mobility events, or updated system information parameters."
    }
]
"""


'''
questions_answers = [
    {
        "question": "How does the UE maintain context while transitioning between RRC states?",
        "answer": "The UE stores essential parameters like RNTI and maintains an active signaling connection, ensuring quick state transitions." 
    },
    {
        "question": "What is the purpose of measurement configurations in 5G networks?",
        "answer": "Measurement configurations define the conditions under which the UE performs signal strength and quality evaluations, ensuring optimal reselection." 
    },
    {
        "question": "Why does the UE rely on periodicity timers for reselection measurements?",
        "answer": "Periodicity timers ensure the UE performs regular checks on neighboring cells without excessive power consumption." 
    },
    {
        "question": "What happens when the UE encounters overlapping PLMN broadcasts?",
        "answer": "The UE prioritizes PLMNs based on stored preferences or user input to select the most suitable network." 
    },
    {
        "question": "What is the function of mobility anchors in inter-RAT handovers?",
        "answer": "Mobility anchors provide seamless connectivity by maintaining data sessions while the UE transitions between RATs." 
    },
    {
        "question": "How does UE behavior change in areas with sparse network coverage?",
        "answer": "The UE increases measurement activities and may camp on acceptable cells to maintain basic connectivity." 
    },
    {
        "question": "What role does NR-ARFCN play in cell selection and reselection?",
        "answer": "NR-ARFCN provides the frequency reference for UE to identify and measure candidate cells during reselection." 
    },
    {
        "question": "How does the UE manage reselection during high-speed mobility?",
        "answer": "The UE uses speed-dependent scaling factors and prioritizes measurements on higher-priority frequencies to reduce reselection latency." 
    },
    {
        "question": "What information is conveyed in SIB1 to guide cell reselection?",
        "answer": "SIB1 provides parameters like cell barring, intra-frequency priority, and thresholds for reselection." 
    },
    {
        "question": "Why is power-saving crucial in idle and inactive modes?",
        "answer": "Power-saving mechanisms like DRX and RNA configuration ensure extended battery life while maintaining connectivity readiness." 
    }
]
'''
'''
questions_answers = [
    {
        "question": "How does the UE evaluate neighboring cell performance during handovers?",
        "answer": "The UE uses signal measurements like RSRP and RSRQ to evaluate and compare neighboring cell performance."
    },
    {
        "question": "What role do beamforming techniques play in 5G connectivity?",
        "answer": "Beamforming improves signal strength and quality by focusing transmission and reception in specific directions."
    },
    {
        "question": "How are priority levels assigned to frequency bands in 5G?",
        "answer": "Priority levels are configured by the network and broadcast through system information to guide UE behavior."
    },
    {
        "question": "What is the significance of the serving cell in NR networks?",
        "answer": "The serving cell provides the primary connection for the UE, facilitating data transfer and control signaling."
    },
    {
        "question": "How does the UE ensure seamless handovers during mobility?",
        "answer": "The UE performs measurements and reports them to the network, which decides and executes the handover process."
    },
    {
        "question": "What parameters affect the UE's paging cycle in idle mode?",
        "answer": "Parameters like DRX cycle length and paging occasion configuration affect how often the UE wakes to check for pages."
    },
    {
        "question": "What happens when a UE fails to complete a handover?",
        "answer": "The UE reverts to the source cell or attempts to reconnect to the network to maintain service continuity."
    },
    {
        "question": "What is the purpose of SIB2 in NR networks?",
        "answer": "SIB2 contains cell-specific configuration details, such as reselection thresholds and access control information."
    },
    {
        "question": "How does the UE identify barred cells during cell selection?",
        "answer": "The UE decodes system information to check for barring status and avoids camping on barred cells."
    },
    {
        "question": "What are the implications of using shared spectrum for UE connectivity?",
        "answer": "Shared spectrum can increase network efficiency but requires the UE to handle dynamic resource allocation and interference."
    }
]
'''
'''
questions_answers = [
    {
        "question": "What mechanisms does the UE use to ensure reliable connectivity during inter-frequency reselection?",
        "answer": "The UE uses thresholds like ThreshX,HighQ and performs measurements on candidate frequencies to ensure seamless reselection."
    },
    {
        "question": "How does the UE handle inter-RAT reselection between 5G and LTE?",
        "answer": "The UE evaluates inter-RAT priority configurations and performs signal quality checks before switching."
    },
    {
        "question": "What factors influence the UE's choice of tracking area?",
        "answer": "Tracking area selection depends on parameters like signal strength, mobility, and network-defined configurations."
    },
    {
        "question": "Why are RAN-based notification areas important in RRC_INACTIVE state?",
        "answer": "They minimize paging overhead by restricting notifications to a subset of cells where the UE is likely located."
    },
    {
        "question": "What happens when a UE detects a barred tracking area?",
        "answer": "The UE avoids camping on cells within the barred tracking area and searches for alternative suitable cells."
    },
    {
        "question": "How does the UE process paging messages for emergency services?",
        "answer": "The UE treats all acceptable cells as suitable and processes emergency paging even on non-registered PLMNs."
    },
    {
        "question": "What criteria are evaluated for reselection in shared spectrum operations?",
        "answer": "The UE considers interference levels, priority configurations, and load balancing metrics."
    },
    {
        "question": "How do mobility states affect reselection timers like TreselectionNR?",
        "answer": "Higher mobility states reduce the timers, allowing faster reselection to adapt to rapid signal changes."
    },
    {
        "question": "What is the role of QoffsetFreq in inter-frequency reselection?",
        "answer": "QoffsetFreq adjusts the measured signal quality of a frequency, influencing its ranking during reselection."
    },
    {
        "question": "Why is signal quality critical in 5G reselection decisions?",
        "answer": "Signal quality metrics like Squal ensure the UE connects to cells with reliable and efficient service levels."
    }
]
'''
'''
questions_answers = [
    {
        "question": "What is the impact of RNA updates in RRC_INACTIVE state?",
        "answer": "RNA updates help maintain UE's context within a specific area, reducing signaling overhead and improving power efficiency."
    },
    {
        "question": "How does the UE prioritize reselection between inter-frequency and intra-frequency cells?",
        "answer": "Intra-frequency cells are prioritized if they fulfill the reselection criteria. Inter-frequency cells are considered if they have higher priority and better signal quality."
    },
    {
        "question": "What role do hysteresis parameters play in cell reselection?",
        "answer": "Hysteresis parameters prevent frequent reselection by adding thresholds, ensuring stability in cell connections."
    },
    {
        "question": "How does the UE handle paging in a multi-PLMN environment?",
        "answer": "The UE listens for paging messages in all tracking areas associated with the selected PLMNs, optimizing response times."
    },
    {
        "question": "What criteria does the UE use to evaluate sidelink communication suitability?",
        "answer": "The UE evaluates parameters like signal strength, quality, and availability of sidelink configuration in system information broadcasts."
    },
    {
        "question": "What happens if a UE fails to decode system information on a selected cell?",
        "answer": "The UE continues searching for another suitable or acceptable cell to camp on, ensuring network availability."
    },
    {
        "question": "How are measurement gaps used to enhance cell reselection accuracy?",
        "answer": "Measurement gaps allow the UE to perform inter-frequency measurements without affecting its ongoing communication, improving reselection decisions."
    },
    {
        "question": "What is the role of PLMN identity in cell reselection?",
        "answer": "PLMN identity ensures the UE selects cells belonging to its registered or equivalent networks, maintaining seamless connectivity."
    },
    {
        "question": "How does the UE handle access control restrictions during cell selection?",
        "answer": "The UE checks access control parameters like CAG identifiers and avoids restricted cells, ensuring compliance with network policies."
    },
    {
        "question": "What is the significance of RSRP thresholds in reselection criteria?",
        "answer": "RSRP thresholds define minimum signal strength requirements for a cell to be considered suitable, ensuring reliable connectivity."
    }
]
'''
'''
questions_answers = [
    {
        "question": "How does the UE adapt to power-saving mechanisms during idle mode?",
        "answer": "The UE utilizes features like Discontinuous Reception (DRX) cycles and monitors only specific paging occasions to save battery power."
    },
    {
        "question": "What happens during inter-frequency reselection when thresholds are met?",
        "answer": "The UE evaluates the target frequency’s quality parameters like Srxlev and Squal before completing the reselection process."
    },
    {
        "question": "How does the UE maintain connectivity in dense urban environments?",
        "answer": "The UE frequently measures and ranks cells to ensure it connects to the most reliable network amidst fluctuating signals."
    },
    {
        "question": "What role does system information play in guiding UE reselection?",
        "answer": "System Information Blocks (SIBs) broadcast critical data like reselection thresholds and frequency priorities to assist UE decision-making."
    },
    {
        "question": "How does the UE avoid frequent reselection events in high-mobility scenarios?",
        "answer": "The UE applies hysteresis margins and speed-based scaling to stabilize connections and reduce unnecessary reselection."
    },
    {
        "question": "What mechanisms ensure reliable handovers between LTE and 5G?",
        "answer": "The UE uses dual connectivity and mobility anchors to manage seamless transitions across RATs without service disruption."
    },
    {
        "question": "Why does the UE periodically recheck the serving cell in idle mode?",
        "answer": "Periodic checks verify that the serving cell still meets quality requirements and allows the UE to detect better alternatives if available."
    },
    {
        "question": "What factors influence the UE’s paging response time?",
        "answer": "Paging response time depends on DRX configurations, the size of the paging group, and network congestion levels."
    },
    {
        "question": "How does the UE determine whether a cell is barred or reserved?",
        "answer": "The UE decodes broadcast information like barred cell identifiers or access control parameters to identify restricted cells."
    },
    {
        "question": "What is the significance of TreselectionEUTRA in 5G systems?",
        "answer": "TreselectionEUTRA ensures that the UE meets specific criteria for stability before moving to an LTE cell during inter-RAT transitions."
    }
]
'''
'''
questions_answers = [
    {
        "question": "What happens when a UE detects overlapping RNAs during RRC_INACTIVE?",
        "answer": "The UE resolves overlapping RNA configurations by selecting the one with the highest priority or signal quality."
    },
    {
        "question": "How does the UE evaluate emergency cell broadcasts?",
        "answer": "Emergency cells are prioritized based on acceptability, allowing the UE to connect even when normal reselection criteria are not met."
    },
    {
        "question": "What role do network slicing configurations play in cell reselection?",
        "answer": "Network slicing provides dedicated resources for specific applications, influencing the UE’s reselection decisions based on slice priority."
    },
    {
        "question": "How does the UE handle dual connectivity between LTE and NR?",
        "answer": "The UE simultaneously maintains connections with both LTE and NR to enhance data throughput and handover reliability."
    },
    {
        "question": "What mechanisms are used to avoid ping-pong reselections?",
        "answer": "Hysteresis and reselection timers like Treselection prevent the UE from oscillating between cells with similar signal conditions."
    },
    {
        "question": "What factors trigger a UE to perform periodic reselection measurements?",
        "answer": "Periodic reselection measurements are triggered by timers or changes in network configurations to ensure the UE is connected to the best cell."
    },
    {
        "question": "How does the UE respond to system information updates in idle mode?",
        "answer": "The UE decodes updated system information and adjusts its reselection parameters or behaviors accordingly."
    },
    {
        "question": "What is the significance of Qrxlevmin in cell reselection?",
        "answer": "Qrxlevmin defines the minimum required signal strength for a cell to be considered suitable, ensuring stable connectivity."
    },
    {
        "question": "How does the UE handle reselection in a network with mixed TDD and FDD deployments?",
        "answer": "The UE evaluates the signal quality and configuration of both TDD and FDD cells, selecting the one that offers better service."
    },
    {
        "question": "What happens when the UE moves to a low-priority PLMN during mobility?",
        "answer": "The UE continues monitoring for higher-priority PLMNs and reselects to them when their criteria are met."
    },
    {
        "question": "How does the UE balance power consumption and reselection accuracy?",
        "answer": "The UE optimizes measurement intervals and uses relaxed thresholds in low-mobility scenarios to balance power usage and accuracy."
    },
    {
        "question": "What role do beam measurements play in NR reselection?",
        "answer": "Beam measurements help the UE identify the strongest beams in an NR cell, improving the quality of its reselection decisions."
    },
    {
        "question": "How does the UE process forbidden tracking area lists during reselection?",
        "answer": "The UE avoids cells in forbidden tracking areas, ensuring compliance with network restrictions."
    },
    {
        "question": "What role does mobility history play in reselection decisions?",
        "answer": "Mobility history allows the UE to prioritize cells or frequencies that have previously provided stable connections."
    },
    {
        "question": "Why is SIB12 crucial in public warning systems?",
        "answer": "SIB12 carries configuration for public warning notifications, ensuring the UE can receive critical alerts."
    }
]
'''
"""
questions_answers = [{
        "question": "What impact does the frequency band have on reselection speed?",
        "answer": "Lower frequency bands typically have better coverage, leading to quicker reselection in areas with poor high-frequency coverage."
    },
    {
        "question": "How does the UE prioritize reselection between public and private networks?",
        "answer": "The UE uses network priority configurations broadcast in system information to decide between public and private networks."
    },
    {
        "question": "What role does backoff timing play in reselection after failed attempts?",
        "answer": "Backoff timing prevents the UE from repeatedly attempting to reselect a cell that previously failed, improving overall network efficiency."
    },
    {
        "question": "How does the UE handle reselection in multi-operator core networks (MOCN)?",
        "answer": "The UE evaluates the PLMN configurations within the shared network and selects based on its operator’s priority."
    },
    {
        "question": "Why is reselection performance crucial in high-speed mobility scenarios?",
        "answer": "Fast and accurate reselection ensures uninterrupted service, especially when the UE is moving quickly between cells."
    }
]
"""
# Combine all question lists
#questions_answers +=   additional_questions
#questions_answers += new_questions



load_dotenv()
llmsan = ChatOpenAI(base_url="https://api.sambanova.ai/v1/",
                 api_key = os.environ['SAMBANOVA_API_KEY'],
                 streaming=True,
                 model = "Meta-Llama-3.2-3B-Instruct")

llm = ChatOllama(
                 model = "llama3.2")


# Sonuçları saklamak için bir liste
evaluation_results = []

# Her bir input üzerinde işlem yapma

for qa_pair in questions_answers:
    question = qa_pair["question"]
    ground_truth = qa_pair["answer"]
    # similarity_search çıktısını stringe dönüştür
    retrieved_contexts = vectorstore.similarity_search(question, k=10)
    combined_context = " ".join([doc.page_content for doc in retrieved_contexts])

    truncated_context = truncate_to_token_limit((combined_context), token_limit=4000)
    input_message = HumanMessage(content=f"Context: {truncated_context}\n\nQuestion: {question}")
    response = llmsan.invoke([input_message])
    # Değerlendirme prompt'unu hazırlama
    grader_prompt = GRADER_PROMPT_TEMPLATE.format(
                    context=retrieved_contexts,
                    question=question,
                    response=response.content,
                    ground_truth=ground_truth
                                                )
    
    # LLM for Evaluation SanBanova
    evaluation_result = llmsan.invoke(grader_prompt)
    
    # Sonuçları saklama
    evaluation_results.append({
        "query": question,
        "response": response,
        "ground_truth": ground_truth,
        "evaluation": evaluation_result.content
    })


In [ ]:
    print(f"Question:{question}")
    print(f"Response:{response}")
    print(f"Ground Truth:{ground_truth}")
    print(f"Context:{evaluation_result.content}")
    print("\n\n\n\n")

In [ ]:
import csv
import os

# Prepare the header for the CSV file
csv_header = [
    "Question", 
    "Ground Truth", 
    "Context", 
    "Groundedness", 
    "Answer Relevance", 
    "Context Relevance"
]

# Define the CSV file path
csv_file_path = "evaluation_results.csv"

# Initialize lists to store scores for average calculation
groundedness_scores = []
answer_relevance_scores = []
context_relevance_scores = []

# Check if the file already exists
file_exists = os.path.exists(csv_file_path)

# Open the CSV file in append mode
with open(csv_file_path, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write the header only if the file is new
    if not file_exists:
        writer.writerow(csv_header)
    
    # Loop through the evaluation results and write each one to the CSV
    for result in evaluation_results:
        # Extract the scores and context from the evaluation result
        evaluation = result["evaluation"]
        
        # Parse the evaluation string to extract the individual scores
        scores = {}
        for line in evaluation.split("\n"):
            if line.startswith("Groundedness"):
                scores["Groundedness"] = float(line.split(":")[1].strip().split("/")[0])  # Extract before "/"
            elif line.startswith("Answer Relevance"):
                scores["Answer Relevance"] = float(line.split(":")[1].strip().split("/")[0])  # Extract before "/"
            elif line.startswith("Context Relevance"):
                scores["Context Relevance"] = float(line.split(":")[1].strip().split("/")[0])  # Extract before "/"

        # Write the row to the CSV
        writer.writerow([
            result["query"], 
            result["ground_truth"], 
            result["response"].content,  # assuming the response content is the string to be saved
            scores.get("Groundedness", ""),
            scores.get("Answer Relevance", ""),
            scores.get("Context Relevance", "")
        ])
        
        # Accumulate the scores for averaging
        groundedness_scores.append(scores.get("Groundedness", 0))
        answer_relevance_scores.append(scores.get("Answer Relevance", 0))
        context_relevance_scores.append(scores.get("Context Relevance", 0))

print(f"Results appended to {csv_file_path}")


In [ ]:
import pandas as pd

# CSV dosyasını yükleyin
df = pd.read_csv('evaluation_results.csv')

# İlgili sütunu yazdırın (örneğin 'sütun_adı' sütunu)

print(df['Context Relevance'].mean())
print(df['Answer Relevance'].mean())